## Построение метода распознавания для определения тех владельцев кредитных карт, которые не вернут кредит в срок. Метод Random Forest

In [14]:
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [15]:
os.chdir("C:/Users/danil/Desktop/Саморазвитие/ГПН/Labs/lab_6_А_К")

In [16]:
df = pd.read_csv('default of credit card clients.csv', sep = ';').drop(columns = ['ID'])  # колонка бесполезная, поэтому удалю её

In [17]:
len(df[df.notna()]) == len(df)  # пропусков нет

True

In [18]:
df['SEX'].value_counts() # по полу дф относительно сбалансирован

2    18112
1    11888
Name: SEX, dtype: int64

In [19]:
df['EDUCATION'].value_counts() # если бы нужно было предсказывать степень обучения человека, то необходима была бы балансировка

2    14030
1    10585
3     4917
5      280
4      123
6       51
0       14
Name: EDUCATION, dtype: int64

In [20]:
df['default_payment_next_month'].value_counts()  # удивительно, что соотношение дефолта и платежа - 1 к 6

0    23364
1     6636
Name: default_payment_next_month, dtype: int64

In [21]:
df

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,1,3,1,39,0,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,150000,1,3,2,43,-1,-1,-1,-1,0,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,30000,1,2,2,37,4,3,2,-1,0,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,80000,1,3,1,41,1,-1,0,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [22]:
X = df.iloc[:, :-1].values
y = df.iloc[:, 23].values

In [23]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 13)  

In [24]:
model = RandomForestClassifier(random_state=13,
                               # число деревьев в лесу
                               n_estimators=80,
                               # функция для impurity ('gini' или 'entropy')
                               criterion='gini',
                               # максимальная глубина дерева
                               max_depth=5,
                               # сколько переменных использовать при построении дерева?
                               # sqrt(k)
                               max_features='sqrt', 
                               # максимальное число конечных узлов. Сейчас - без ограничений
                               max_leaf_nodes=None,
                               # минимальное уменьшение загрязнения 
                               min_impurity_decrease=0.001, 
                               # Дерево обучается по части наблюдений
                               bootstrap=True, 
                               # минимальное число наблюдений в узле потомке
                               min_samples_leaf=10, 
                               # минимальное число наблюдений в узле родителе
                               min_samples_split=15,
                               # параметр используется редко, самостоятельно
                               min_weight_fraction_leaf=0.0, 
                               # объем промежуточных результатов в ходе обучения
                               verbose=1, 
                               # число используемых процессоров
                               # n_jobs=None,
                               # Вычислять out-of-bag ошибку
                               oob_score=True,
                               # использовать результаты предыдущего вызова и нарастить предыдущий лес 
                               warm_start=False,
                               # веса классов для балансировки обучения
                               class_weight=None)

max_samples = 0.66

model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.7s finished


RandomForestClassifier(max_depth=5, max_features='sqrt',
                       min_impurity_decrease=0.001, min_samples_leaf=10,
                       min_samples_split=15, n_estimators=80, oob_score=True,
                       random_state=13, verbose=1)

In [25]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.0s finished


In [26]:
importance_df = pd.DataFrame({'feature': df.iloc[:, :-1].columns,
              'importance': model.feature_importances_}).sort_values('importance', ascending=False)
importance_df[importance_df['importance']==0]['feature'].tolist()  # вывожу в список показатели, полностью не влияющие на модель  
# экспериментально подтвердилось, что в случае удаления этих колонок основные показатели качества модели не снижаются

['AGE', 'MARRIAGE', 'EDUCATION']

In [27]:
from sklearn.metrics import accuracy_score

print("Accuracy на обучающем множестве ", accuracy_score(y_train, y_pred_train)*100)
print("Accuracy на тестовом множестве ", accuracy_score(y_test, y_pred_test)*100)

Accuracy на обучающем множестве  81.4726368159204
Accuracy на тестовом множестве  82.01010101010101


In [28]:
print(metrics.classification_report(y_test,  y_pred_test))

              precision    recall  f1-score   support

           0       0.83      0.96      0.89      7736
           1       0.70      0.30      0.43      2164

    accuracy                           0.82      9900
   macro avg       0.77      0.63      0.66      9900
weighted avg       0.80      0.82      0.79      9900



In [30]:
from sklearn.model_selection import GridSearchCV
param_grid = {
     'n_estimators': [c for c in range(30,151,10)],
     'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
     "min_samples_split": [c for c in range(3, 12, 2)]}

grid_search_res = GridSearchCV(model, param_grid=param_grid, cv=4, 
                               #  выбор критерия качества
                               scoring='accuracy', 
                               #  поведение в случае ошибки при вычислении критерия качества
                               error_score='raise', 
                               #  обучить модель с лучшими значениями параметров
                               refit=True, 
                               pre_dispatch=None, verbose=0)

In [ ]:
grid_search_res.fit(X_train, y_train)

In [41]:
#   Лучший набор значений гиперпараметров

best_hyperparams = grid_search_res.best_params_
print('Лучшие значения гиперпараметров:\n', best_hyperparams)

Лучшие значения гиперпараметров:
 {'max_depth': 5, 'min_samples_split': 3, 'n_estimators': 40}


In [42]:
# Значение критерия качества для лучших значений гиперпараметров 
best_CV_score = grid_search_res.best_score_
print('Лучшее accuracy', best_CV_score)

Лучшее accuracy 0.8148258706467661


In [43]:
# Лучшая модель
best_model = grid_search_res.best_estimator_

# accuracy на тестовом множестве для лучшей модели
test_acc = best_model.score(X_test,y_test)

# Значение критерия качества для лучших значений гиперпараметров 
print('Лучшее accuracy на тестовом множестве', test_acc)
print('Лучшее accuracy на тестовом множестве CART – 0.8219')
print('Лучшее accuracy на тестовом множестве методом KNN – 0.8065')

Лучшее accuracy на тестовом множестве 0.8202020202020202
Лучшее accuracy на тестовом множестве CART – 0.8219
Лучшее accuracy на тестовом множестве методом KNN – 0.8065


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


In [44]:
# Строим предсказание модели
y_pred_train = grid_search_res.predict(X_train)
y_pred_test = grid_search_res.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


In [45]:
from sklearn import metrics
conf_mat = metrics.confusion_matrix(y_test, y_pred_test)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,0,1
0,7446,290
1,1490,674


Результаты матрицы ошибок на тестовой выборке
- 7446 - True Positive (истинно-положительное решение): прогноз совпал с реальностью, результат положительный произошел, как и было предсказано ML-моделью  vs 7425 CART

- 290 - False Positive (ложноположительное решение): ошибка 1-го рода, ML-модель предсказала положительный результат, а на самом деле он отрицательный vs 311 CART

- 1490 - False Negative (ложноотрицательное решение): ошибка 2-го рода – ML-модель предсказала отрицательный результат, но на самом деле он положительный vs 1452 CART

- 674 - True Negative (истинно-отрицательное решение): результат отрицательный, ML-прогноз совпал с реальностью vs 712 CART

In [46]:
conf_mat = metrics.confusion_matrix(y_train, y_pred_train)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,0,1
0,14996,632
1,3065,1407


In [47]:
# Строим предсказание модели
y_pred_train2 = grid_search_res.predict_proba(X_train) # вероятность принадлежать тому или иному классу
y_pred_test2 = grid_search_res.predict_proba(X_test)
print(y_pred_train2)
print(y_pred_test2)

[[0.76702904 0.23297096]
 [0.84575281 0.15424719]
 [0.82633397 0.17366603]
 ...
 [0.89250581 0.10749419]
 [0.69415287 0.30584713]
 [0.84318101 0.15681899]]
[[0.73615819 0.26384181]
 [0.89297007 0.10702993]
 [0.8830033  0.1169967 ]
 ...
 [0.87417329 0.12582671]
 [0.77290124 0.22709876]
 [0.82996334 0.17003666]]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


In [48]:
y_pred_train2 = grid_search_res.predict(X_train)
y_pred_test2 = grid_search_res.predict(X_test)
print(metrics.classification_report(y_pred_test2, y_test))

              precision    recall  f1-score   support

           0       0.96      0.83      0.89      8936
           1       0.31      0.70      0.43       964

    accuracy                           0.82      9900
   macro avg       0.64      0.77      0.66      9900
weighted avg       0.90      0.82      0.85      9900



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


Таким образом, была построена модель с уровнем accuracy примерно совпадающим с моделью CART. 
Важно заметить разницу в матрице ошибок по сравнению с методом CART:
1. Больше TP и FN (ошибок 2 рода)
2. Меньше TN и FP (ошибок 1 рода)